In [1]:
import os
os.chdir('../../src/')
from visualization.SIR_Plot import make_SIR_graph, make_beds_graph

In [2]:
os.chdir('models')
import simulate_pandemic as simp
from actions import costs, city_restrictions
from MCFS import mcts, treeNode
from CMDP import CovidState
from glob import glob

In [5]:
import plotly.graph_objects as go
import pandas as pd

def make_beds_graph(data, actions, step_size, title, max_range=None, color_map=None, make_df=True):
    fig = go.Figure()
    
    if make_df:
        data = pd.DataFrame([pd.Series(d).value_counts() for d in data] + [pd.Series(data[-1]).value_counts()])
        data.fillna(0, inplace=True)
        data

    color_map = {
        'Lockdown':          'rgb(0.83, 0.13, 0.15)',
        'Hard Quarantine':    'rgb(0.85, 0.35, 0.13)',
        'Light Quarantine':   'rgb(0.97, 0.91, 0.56)',
        'Social Distancing':  'rgb(0.67, 0.88, 0.69)',
        'Unrestricted':        'rgb(0.86, 0.86, 0.86)'    
    }

    pop = 55492
    
    actions = list(map(color_map.get,  actions))

    x = list(range(len(data) + 1))
    
    #\definecolor{royalblue(web)}{rgb}{0.25, 0.41, 0.88}
    fig.add_trace(go.Scatter(x=x, y=data['hospitalized']/pop, name='hospitalized', line_color = 'rgb(0.25, 0.41, 0.88)',
                            line=dict(width=3.5),  
                            # yaxis="y2"
                            ))
    #\definecolor{firebrick}{rgb}{0.7, 0.13, 0.13}
    #fig.add_trace(go.Scatter(x=x, y=data[1]/pop, name='exposed', line_color = 'rgb(0.7, 0.13, 0.13)',
    #                        line=dict(width=4)))
    
    fig.add_trace(go.Scatter(x=x, y=(len(data)+1)*[0.0015], name='capacity', line_color = 'black',
                            line=dict(dash='dash', width = 2),
                            # yaxis="y2"
                            ))
    fig.update_layout(
        shapes=[
            dict(
                type="rect",
                # x-reference is assigned to the x-values
                xref="x",
                # y-reference is assigned to the plot paper [0,1]
                yref="paper",
                x0=x[step_size*i],
                y0=0,
                x1=x[step_size*(i+1)],
                y1=1,
                fillcolor=a,
                opacity=0.45,
                layer="below",
                line_width=0,
            ) for i,a in enumerate(actions)] 
    )

    for k,v in color_map.items():
        fig.add_trace(go.Bar(x=[None], y=[None], marker=dict(color=v), name = k))

    fig.update_layout(xaxis={'showgrid': False,},
                      yaxis = {'showgrid': False, 'zeroline': False,  'title': 'Proportion of Population Hospitalized'},
                      showlegend=True, hovermode="x")
    
    fig.update_layout(
        xaxis = dict(
            tickmode = 'array',
            tickvals = list(range(0, len(data)+1, 14)),
            ticktext = list(range(0, int(len(data)/7) + 1, 2)),
            title = 'Weeks'
            )
        )
    fig.update_layout(showlegend=False,
        margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=0
    ))
    
    """fig.update_layout(
        yaxis=dict(
            title="Proportion of Population Exposed",
            titlefont=dict(
                color="rgb(0.7, 0.13, 0.13)"
            ),
            showgrid=False,
            tickfont=dict(
                color="rgb(0.7, 0.13, 0.13)"
            ),
            #range=[0, max_range*10]

        ),
        yaxis2=dict(
            title="Proportion of Population Hospitalized",
            titlefont=dict(
                color="rgb(0.25, 0.41, 0.88)"
            ),
            tickfont=dict(
                color="rgb(0.25, 0.41, 0.88)"
            ),
            showgrid=False,
            anchor="x",
            overlaying="y",
            side="right",
            #range=[0, max_range]

        ),)"""

    fig.update_yaxes(automargin=True)
    fig.update_layout(title=title)
    fig.write_image(f"{title}.pdf")

    fig.show()
    
    return data

In [8]:
import pickle as pkl

for file in glob('../../data/MCTS_Results/pickles/optimized_*'):
    with open(file, 'rb') as f:
        data, ts, actions, tree = pkl.load(f)
        df = make_beds_graph(ts,
                             actions[:-1], 
                             7,
                             file.split('optimized_')[1],
                             make_df=False)

In [28]:
ts

,removed,susceptible,exposed,infected,hospitalized
0,0.0,55460.0,32.0,0.0,0.0
1,0.0,55458.0,33.0,1.0,0.0
2,0.0,55456.0,31.0,5.0,0.0
3,0.0,55454.0,30.0,8.0,0.0
4,0.0,55449.0,28.0,15.0,0.0
...,...,...,...,...,...
359,41256.0,14173.0,20.0,37.0,6.0
360,41261.0,14168.0,22.0,35.0,6.0
361,41264.0,14164.0,20.0,38.0,6.0
362,41269.0,14159.0,23.0,35.0,6.0


In [11]:
leafs = ['a', 'b', 'c', 'd']

[leaf for leaf in leafs for i in range(3)]

['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'c', 'd', 'd', 'd']

In [26]:
tree.root.action

AttributeError: 'treeNode' object has no attribute 'action'

In [15]:
level1 = tree.root.children
len(level1)

5

In [21]:
len([kk for k,v in tree.root.children.items() for kk,vv in v.children.items()])

13

In [23]:
len([kkk for k,v in tree.root.children.items() 
         for kk,vv in v.children.items()
         for kkk,vvv in vv.children.items()])

35

In [25]:
len([kkk for k,v in tree.root.children.items() 
         for kk,vv in v.children.items()
         for kkk,vvv in vv.children.items()
         for kkkk,vvvv in vvv.children.items()
    ])

95

In [ ]:
from sklearn.metrics import mean_absolute_error

pop=55e3
mean_absolute_error(y_true=len(df.iloc[4:])*[0.0015],
                    y_pred=df['hospitalized'].iloc[4:]/pop)

In [ ]:
import pickle as pkl

with open('time_policy_2.pkl', 'rb') as f:
    results = pkl.load(f)

In [ ]:
results[0]